Before starting let's go back to out main directory. In USERNAME write the user path you are using...

In [ ]:
import os 
os.chdir('C:/Users/USERNAME/')

Welcome to the calibration demo.
Before you begin, obtain a rigid checkerboard with an odd number of internal corners (e.g., 9×7 or 10×8). If you don’t have one, cover some of the squares on a regular checkerboard with white paper to achieve the desired pattern.
Remember that the dimensions used by the algorithms refer to the number of internal corners, which is always one less than the number of squares. For example, a 10×8 board will be interpreted as a 9×7 pattern, and so on.
Next, measure the size of a single square on your checkerboard and convert that length into meters—you will need this value later.
Recording Your Calibration Videos
1 Record synchronized video from your multi-camera setup, making sure the checkerboard is clearly visible in the first frame for each camera you plan to calibrate.  
2 Save these videos and rename them according to the camera names you’ve chosen—ideally reflecting each camera’s position in 3D space.  
3 Ensure all videos are:
   - Synchronized (no more than 2–3 frames difference between cameras)  
   - Short (no longer than 1 minute)  
   - Recorded at the same frame rate (on streams7, adjust each camera’s internal frame drop to match).

You can find an example of calibration videos in the various_data folder...

Creating the Calibration Folders
Now let’s create the directory structure we’ll use for calibration by invoking the following function:

In [ ]:
!python EasyMocap/apps/hopeitworks/creacartelle.py

Now proceed to upload the videos (e.g. CAM1.mp4 CAM2.mp4) into the videos folders of both extri_data and intri_data

PS If you ever want to clear the existing data in the two folders without deleting the videos, feel free to use the functions provided below by uncommenting them (removing the #)

In [ ]:
#!python EasyMocap/apps/hopeitworks/cleanintri.py

In [ ]:
#!python EasyMocap/apps/hopeitworks/cleanextri.py

PS Obvioulsy we have to create again the auxilary folders..

In [ ]:
!python EasyMocap/apps/hopeitworks/creacartelle.py

Ok, now we can proceed with image extraction and chessboard detection using the following commands; of course, before doing so, change pattern and grid to your own values:

In [ ]:
##INTRINSECO
# Estrai le immagini per il dataset intrinseco
!python EasyMocap/scripts/preprocess/extract_video.py EasyMocap/intri_data --no2d
# Rileva la scacchiera per il dataset intrinseco
!python EasyMocap/apps/calibration/detect_chessboard.py EasyMocap/intri_data --out EasyMocap/intri_data --pattern 9,7 --grid 0.05
##ESTRINSECO
# Estrai le immagini per il dataset estrinseco
!python EasyMocap/scripts/preprocess/extract_video.py EasyMocap/extri_data --no2d
# Rileva la scacchiera per il dataset estrinseco
!python EasyMocap/apps/calibration/detect_chessboard.py EasyMocap/extri_data --out EasyMocap/extri_data --pattern 9,7 --grid 0.05

Let’s move on to camera calibration—first intrinsic, then extrinsic—using the following functions:

In [ ]:
# Calibrazione dei parametri intrinseci
!python EasyMocap/apps/calibration/calib_intri.py EasyMocap/intri_data
# Calibrazione dei parametri estrinseci
!python EasyMocap/apps/calibration/calib_extri.py EasyMocap/extri_data --intri EasyMocap/intri_data/output/intri.yml
##Semplice check con stampa CUBO
!python EasyMocap/apps/calibration/check_calib.py EasyMocap/extri_data --out EasyMocap/extri_data --mode cube --write

Warning! This step may produce errors if OpenCV’s findChessboardCorners fails to detect the corners. Once you’ve confirmed the error is due to this, record new videos, trying different checkerboard angles.
PS: the lower the checkerboard is relative to the camera plane, the higher the chance of success.

Once you have some results, we can move on to the final step. 
Open the cube folder in extri_data..you should see images from each camera. 
Now take a close look at the cube projected in all four photos.

OPTION A: The projected cube looks distorted… you must have made an error in one of the previous steps. Try to fix it—good luck.

OPTION B: The cube is projected correctly… perfect! You’ve generated extri.yml and intri.yml, the files containing your cameras’ calibration data.

OPTION C (most likely): The cube projects correctly, but the corner adjacent to the checkerboard appears in different positions depending on the camera.

OPTION C Resolution.
Chances are your cameras will project the vertex to two distinct locations. 

Let's call these 2 positions X and Y..

To resolve this, we will only adjust the cameras that place it at the point we’ll call X. Ignore those that place the vertex at point Y.
To proceed, complete the cell below by replacing CAM1 and CAM2 with your video names (i.e., the camera names without .mp4) and run the cell.

In [ ]:
!python EasyMocap/apps/hopeitworks/ruotajsonintri.py CAM1
!python EasyMocap/apps/hopeitworks/ruotajsonextri.py CAM1
!python EasyMocap/apps/hopeitworks/ruotajsonintri.py CAM2
!python EasyMocap/apps/hopeitworks/ruotajsonextri.py CAM2

## add others if needed

#RICALIBRAZIONE...con dati aggiornati
!python EasyMocap/apps/calibration/calib_intri.py EasyMocap/intri_data
!python EasyMocap/apps/calibration/calib_extri.py EasyMocap/extri_data --intri EasyMocap/intri_data/output/intri.yml
!python EasyMocap/apps/calibration/check_calib.py EasyMocap/extri_data --out EasyMocap/extri_data --mode cube --write

Now check again the cube folder; If the cube is rendered correctly and all cameras share the same relative vertex position, congratulations—you’ve completed the calibration! In the next steps, we will see how the .yml files are used.
Before closing, please copy the calibration files in their folder with the cell down below...
See you in the next tutorial :)

In [ ]:
import shutil

shutil.copy('EasyMocap/extri_data/extri.yml', 'EasyMocap/yamls')
shutil.copy('EasyMocap/extri_data/intri.yml', 'EasyMocap/yamls')